In [ ]:
___Author___='LumberJack Jyss'
print('Global Optimized LumberJack Environment Motor\n°OoO_FXCM_Oo0°\nLumberJack Jyss 5779(c)')
print('Version v2.01')
print('Importing librairies...')
import forexconnect
from forexconnect import fxcorepy
from forexconnect import ForexConnect, ResponseListener, Common
import time
import sys
import numpy as np
import pandas as pd
from colorama import Fore, Back, Style 
import datetime as dt
import talib
print('Librairies imported')
import warnings
warnings.filterwarnings("ignore")
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)


def session_status_changed(session: fxcorepy.O2GSession,
                           status: fxcorepy.AO2GSessionStatus.O2GSessionStatus):
    print("Trading session status: " + str(status))
    return



pos_long = 0
pos_short = 0

# TOKEN = 'e053ac1597cef331df9429ac8151100ea9f1c411'
# server = 'demo'
user_id = 'D261151986'
password = '2786'
url = 'fxcorporate.com/Hosts.jsp'
accountid='01147539'
# minutes: 'm1' , 'm5' , 'm15' , 'm30' 
# hours: 'H1' , 'H2', 'H3', 'H4', 'H6' 'H8'
# one day: 'D1'
# one week: 'W1'
# one month: 'M1'
ticker = 'SPX500'
period = 'm1'
number = 30
jet_lag = -2
amount = 30
buysell = None
side = None
trade_id = None
open_rate = None
open_time = None
offerid = None
buy_price = None
sell_price = None
pnl = None

# Time Windows
# start = dt.datetime(2017, 7, 15)
# stop = dt.datetime(2017, 8, 1)
# con.get_candles('EUR/USD', period='D1',start=start, stop=stop)

# CONNEXION

fx = ForexConnect()
fx.login(user_id, password, url,"Demo", session_status_callback=session_status_changed)
table_manager = fx.table_manager
trades_table = table_manager.get_table(fx.TRADES)

# RECOVERING TEST
for row in trades_table:
    trade =None
    side = row['buy_sell']
    pnl = row['pl']
    trade_id = row['trade_id']
    open_rate = row['open_rate']
    should_close = row['close']
    amount = row['amount']
if side == 'B':
    pos_long = 1
elif side == 'S':
    pos_short = 1

# RECCUPERATION DES 30 DERNIERES DONNEES
tmps1=time.time()
history = fx.get_history(ticker, period,quotes_count=number)
data = pd.DataFrame({'Date':history['Date'],'Open':history['BidOpen'],'High':history['BidHigh'],'Low':history['BidLow'],'Close':history['BidClose']})
df = data.copy()
tmps2=round(time.time()-tmps1,2)
print('Data scraped in ',tmps2,' seconds')

print('\033[33m' + 'Initializing GOLEM...')
print('\033[35m' +'Long Pos =',pos_long)
print('Short Pos =',pos_short,Style.RESET_ALL)

# DATA PREPARATION

tmps1=time.time()
print('df.shape control : ',df.shape)
df.reset_index(inplace=True,drop=True)
dataset_1D = df.copy()
dataset_1D = dataset_1D.dropna()
rsi = talib.RSI(dataset_1D['Close'],timeperiod=2)
aroondown, aroonup = talib.AROON(dataset_1D['High'], dataset_1D['Low'], timeperiod=25)
aroondelta = aroonup - aroondown #(aroonup-aroondown)/abs((aroonup-aroondown))
dataset_1D['RSI'] = rsi
dataset_1D['Aroon Up'] = aroonup
dataset_1D['Aroon Down'] = aroondown
dataset_1D['Delta Aroon'] = aroondelta

dataset_1D['target_value'] = dataset_1D['Close']
dataset_1D = dataset_1D.drop(['Open','High','Low','Close'],axis=1)
df_1D = dataset_1D.copy()
df_1D = df_1D.iloc[:,1:]
df_1D['Date'] = df['Date']
df_1D = df_1D.dropna()
df_1D = df_1D[['Date'] + df_1D.columns[:-1].tolist()]
df_1D.reset_index(inplace=True,drop=True)
tmps2=round(time.time()-tmps1,2)
print('Data prepared in ',tmps2,' seconds')

print('\033[36m'+'GOLEM begins Computing...'+Style.RESET_ALL)


print('Computing done')
print('Instrument :', ticker)
print(df_1D.tail())

print('\n Initializing GOLEM on STREAMING LIVE...\n')

# UPDATING DATA
for loop in range(0,200000):
    #print(' \n INFRA LOOP ',loop,'\n')
    tmps1=time.time()

    history = fx.get_history(ticker, period,quotes_count=1)
    last = pd.DataFrame({'Date':history['Date'],'Open':history['BidOpen'],'High':history['BidHigh'],'Low':history['BidLow'],'Close':history['BidClose']})
    
    if last.iloc[-1,0].minute == df.iloc[-1,0].minute :
    
        print('\r \033[43m '+period+' \033[0m'+'\033[46m'+'...'+'\033[0m',end='')
        print ('\033[36m', dt.datetime.now().hour,'H',dt.datetime.now().minute,'min',dt.datetime.now().second,'sec',Style.RESET_ALL,end='')
        if pos_long == 1:
            try:
                if pnl > 0 :
                    print('\033[42m  - pnl : ',round(pnl,2),'\033[0m',end='', flush=True)
                elif pnl < 0:
                    print('\033[41m  - pnl : ',round(pnl,2),'\033[0m',end='', flush=True)
            except:
                pass
        elif pos_short == 1:
            try:
                if pnl > 0 :
                    print('\033[42m  - pnl : ',round(pnl,2),'\033[0m',end='', flush=True)
                elif pnl < 0:
                    print('\033[41m  - pnl : ',round(pnl,2),'\033[0m',end='', flush=True)
            except:
                pass
        else:
            print(' - No position',end='',flush=True)
        time.sleep(0.1)
    else :
        tmps1=time.time()
        df = df.iloc[1:,:]
        df = df.append({'Date':last.iloc[-1,0],'Open':last.iloc[-1,1], 'High':last.iloc[-1,2],'Low':last.iloc[-1,3], 'Close':last.iloc[-1,4]}, ignore_index=True)

        print('df.shape control : ',df.shape)
        df.reset_index(inplace=True,drop=True)
        dataset_1D = df.copy()
        dataset_1D = dataset_1D.dropna()
        rsi = talib.RSI(dataset_1D['Close'],timeperiod=2)
        aroondown, aroonup = talib.AROON(dataset_1D['High'], dataset_1D['Low'], timeperiod=25)
        atr = talib.ATR(dataset_1D['High'],dataset_1D['Low'],dataset_1D['Close'],timeperiod=14)
        aroondelta = aroonup - aroondown #(aroonup-aroondown)/abs((aroonup-aroondown))
        dataset_1D['RSI'] = rsi
        dataset_1D['Aroon Up'] = aroonup
        dataset_1D['Aroon Down'] = aroondown
        dataset_1D['Delta Aroon'] = aroondelta
        dataset_1D['target_value'] = dataset_1D['Close']
        dataset_1D = dataset_1D.drop(['Open','High','Low','Close'],axis=1)
        df_1D = dataset_1D.copy()
        df_1D = df_1D.iloc[:,1:]
        df_1D['Date'] = df['Date']
        df_1D = df_1D.dropna()
        df_1D = df_1D[['Date'] + df_1D.columns[:-1].tolist()]
        df_1D.reset_index(inplace=True,drop=True)
        
        
        print()
        print('\r GOLEM begins testing...',end='')
        print('   Aroon up :',aroonup.iloc[-1],end='')
        print('   Aroon down : ',aroondown.iloc[-1],end='')
        print('   RSI : ',rsi.iloc[-1],end='')
        print ('\033[36m    ', dt.datetime.now().hour,'H',dt.datetime.now().minute,'min',dt.datetime.now().second,'sec',Style.RESET_ALL,end='')
        
        if aroonup.iloc[-1] < 4 and aroondown.iloc[-1] > 97  and rsi.iloc[-1] < 10:

            if  pos_long == 0 and pos_short == 1 :
                
                buysell = 'B'
                try:
                    for row in trades_table:
                        trade =None
                        side = row['buy_sell']
                        pnl = row['pl']
                        trade_id = row['trade_id']
                        open_rate = row['open_rate']
                        should_close = row['close']
                        amount = row['amount']

                    trade_rate = float(fx.get_history(ticker, 't1',quotes_count=1)['Bid'][0]) + (4*float(atr.iloc[-1]))
                    request_close = fx.create_order_request(order_type='CM',
                                                                            ACCOUNT_ID=accountid,
                                                                            OFFER_ID = offerid,
                                                                            BUY_SELL=buysell,
                                                                            AMOUNT=amount,
                                                                            TIME_IN_FORCE='IOC',
                                                                            TRADE_ID = trade_id)
                    resp_close = fx.send_request(request_close)

                    closed_table = table_manager.get_table(fx.CLOSED_TRADES)
                    for row in closed_table:
                        trade =None
                        side = row['buy_sell']
                        pnl = row['pl']
                        trade_id = row['trade_id']
                        open_rate = row['open_rate']
                        amount = row['amount']
                        open_time = row['open_time']
                        close_rate = row['close_rate']
                        close_time = row['close_time']

                    print('\033[47m \033[31m Close Short avec un pnl de', pnl,'\033[0m')
                    print('Résumé : ')
                    print(' Open date : ', open_time)
                    print('Open Rate : ',open_rate)
                    print('Amount : ',amount)
                    print('Close date : ',close_time)
                    print('Close Rate : ',close_rate)
                    print('')
                    pos_short = 0
                    continue
                except:
                    print("WARNING! It wasn't possible to close the position" )
                    pass

            if pos_long == 0 and pos_short == 0:
                
                buysell = 'B'
                trade_rate = float(fx.get_history(ticker, 't1',quotes_count=1)['Bid'][0]) + (4*float(atr.iloc[-1]))
                print('Trying Long at :',trade_rate)
                try:
                    request_open = fx.create_order_request(\
                        SYMBOL=ticker,\
                        order_type='OM',\
                        ACCOUNT_ID=accountid,\
                        BUY_SELL=buysell,\
                        AMOUNT=amount,\
                        TIME_IN_FORCE='IOC'\
                        )
                    resp_open = fx.send_request(request_open)
                    order_id = resp_open.order_id
                    offer = Common.get_offer(fx,ticker)
                    offerid = offer.offer_id
                    table_manager = fx.table_manager
                    trades_table = table_manager.get_table(fx.TRADES)
                    for row in trades_table:
                        trade =None
                        side = row['buy_sell']
                        pnl = row['pl']
                        trade_id = row['trade_id']
                        open_rate = row['open_rate']
                        should_close = row['close']
                        amount = row['amount']
                    print('Side',side,'pnl : ',pnl,'Trade_ID : ',trade_id,'Open_Rate',open_rate,'Amount',amount)

                    print('Résumé : ')
                    print(' Open date : ', open_time)
                    print('Open Rate : ',open_rate)
                    print('Amount : ',amount)
                    print('pnl : ',pnl)
                    print('')
                    print(                                                    'Go Long!')
                    pos_long = 1
                    
                except:
                    print("It wasn't possible to long... What a good opportunity missed... Meoow yells the black cat")
                    pass

        elif aroondown.iloc[-1] < 4 and aroonup.iloc[-1] > 97  and rsi.iloc[-1] > 90:
            
            if  pos_long == 1 and pos_short == 0 :
                
                buysell = 'S'
                try:
                    for row in trades_table:
                        trade =None
                        side = row['buy_sell']
                        pnl = row['pl']
                        trade_id = row['trade_id']
                        open_rate = row['open_rate']
                        should_close = row['close']
                        amount = row['amount']
                    trade_rate = float(fx.get_history(ticker, 't1',quotes_count=1)['Bid'][0]) + (4*float(atr.iloc[-1]))
                    request_close = fx.create_order_request(order_type='CM',
                                                                            ACCOUNT_ID=accountid,
                                                                            OFFER_ID = offerid,
                                                                            BUY_SELL=buysell,
                                                                            AMOUNT=amount,
                                                                            TIME_IN_FORCE='IOC',
                                                                            TRADE_ID = trade_id)
                    resp_close = fx.send_request(request_close)

                    closed_table = table_manager.get_table(fx.CLOSED_TRADES)
                    for row in closed_table:
                        trade =None
                        side = row['buy_sell']
                        pnl = row['pl']
                        trade_id = row['trade_id']
                        open_rate = row['open_rate']
                        amount = row['amount']
                        open_time = row['open_time']
                        close_rate = row['close_rate']
                        close_time = row['close_time']

                    print('\033[47m \033[31m Close Short avec un pnl de', pnl,'\033[0m')
                    print('Résumé : ')
                    print(' Open date : ', open_time)
                    print('Open Rate : ',open_rate)
                    print('Amount : ',amount)
                    print('Close date : ',close_time)
                    print('Close Rate : ',close_rate)
                    print('')
                    pos_long = 0
                    continue
                except:
                    print("WARNING! It wasn't possible to close the position" )
                    continue

            if pos_long == 0 and pos_short == 0:
                
                buysell = 'S'
                trade_rate = float(fx.get_history(ticker, 't1',quotes_count=1)['Bid'][0]) - (4*float(atr.iloc[-1]))
                print('Trying short at : ',trade_rate)
                try:
                    request_open = fx.create_order_request(\
                        SYMBOL=ticker,\
                        order_type='OM',\
                        ACCOUNT_ID=accountid,\
                        BUY_SELL=buysell,\
                        AMOUNT=amount,\
                        TIME_IN_FORCE='IOC'\
                        )
                    resp_open = fx.send_request(request_open)
                    order_id = resp_open.order_id
                    offer = Common.get_offer(fx,ticker)
                    offerid = offer.offer_id
                    table_manager = fx.table_manager
                    trades_table = table_manager.get_table(fx.TRADES)
                    for row in trades_table:
                        trade =None
                        side = row['buy_sell']
                        pnl = row['pl']
                        trade_id = row['trade_id']
                        open_rate = row['open_rate']
                        should_close = row['close']
                        amount = row['amount']
                    print('Side',side,'pnl : ',pnl,'Trade_ID : ',trade_id,'Open_Rate',open_rate,'Amount',amount)

                    print('Résumé : ')
                    print(' Open date : ', open_time)
                    print('Open Rate : ',open_rate)
                    print('Amount : ',amount)
                    print('pnl : ',pnl)
                    print('')
                    print(                                                    'Go Short!')
                    pos_short = 1
                except:
                    print("It wasn't possible to short. You missed a great opportunity!")
                    pass
        else:
            print('No signal at the moment')

Global Optimized LumberJack Environment Motor
°OoO_FXCM_Oo0°
LumberJack Jyss 5779(c)
Version v2.01
Importing librairies...
Librairies imported
Trading session status: CONNECTING
Trading session status: CONNECTED
Data scraped in  0.66  seconds
Initializing GOLEM...
Long Pos = 0
Short Pos = 1 
df.shape control :  (30, 5)
Data prepared in  0.03  seconds
GOLEM begins Computing...
Computing done
Instrument : SPX500
                 Date        RSI  Aroon Up  Aroon Down  Delta Aroon  \
0 2019-07-04 12:18:00  16.796296      28.0       100.0        -72.0   
1 2019-07-04 12:19:00   7.031040      24.0       100.0        -76.0   
2 2019-07-04 12:20:00  67.825562      20.0        96.0        -76.0   
3 2019-07-04 12:21:00  67.825562      16.0        92.0        -76.0   
4 2019-07-04 12:22:00  90.577072      12.0        88.0        -76.0   

   target_value  
0       2996.24  
1       2996.08  
2       2996.34  
3       2996.34  
4       2996.58  

 Initializing GOLEM on STREAMING LIVE...

  m1 ...